In [5]:

import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, RepeatedKFold, LeaveOneOut
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_predict
import numpy as np, seaborn as sns, matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os
import sys

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

# Add project root to path (for Jupyter notebooks)
# Get the current directory and navigate to project root
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.insert(0, project_root)

from src.config import Config
import src.eda as eda
import src.present_value as present_value

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
## FROM DATABASE
pv = present_value.PresentValue()
anual_increment = pv.fetch_incremento_from_database()

fase = "III"
preproccesing = eda.EDA()
df_raw = preproccesing.assemble_projects_from_database(fase)
df_vp = preproccesing.create_dataset(pv.present_value_costs, fase=fase)

c:\Code\Road-Project-Prediction\src\eda.py:292: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  w = (df[cols] / totals).fillna(0)


In [10]:
df_vp[df_vp['9 - TÚNELES']!=0]

,NOMBRE DEL PROYECTO,CÓDIGO,LONGITUD KM,PUENTES VEHICULARES UND,PUENTES VEHICULARES M2,PUENTES PEATONALES UND,PUENTES PEATONALES M2,TUNELES UND,TUNELES KM,ALCANCE,...,7 - SOCAVACIÓN,8 - ESTRUCTURAS,9 - TÚNELES,10 - URBANISMO Y PAISAJISMO,11 - PREDIAL,12 - IMPACTO AMBIENTAL,13 - CANTIDADES,14 - EVALUACIÓN SOCIOECONÓMICA,15 - OTROS - MANEJO DE REDES,16 - DIRECCIÓN Y COORDINACIÓN
20,TERMINACIÓN TÚNEL DE LA LÍNEA,0347801,8.35,10,24258,0,0,5,0.9200,Rehabilitación,...,6.284023e+07,4.415680e+08,6.644646e+07,0.0,1.663940e+07,0.0,0.0,0.0,0.0,2.006346e+08
21,TERMINACIÓN TÚNEL DE LA LÍNEA,0347801,8.60,0,0,0,0,1,8.6000,Rehabilitación,...,6.472167e+07,4.547886e+08,1.632824e+08,0.0,1.713759e+07,0.0,0.0,0.0,0.0,2.066416e+08
22,TERMINACIÓN TÚNEL DE LA LÍNEA,0347801,8.50,16,26811,0,0,15,2.2747,Rehabilitación,...,6.396909e+07,4.495003e+08,1.906902e+08,0.0,1.693831e+07,0.0,0.0,0.0,0.0,2.042388e+08


In [ ]:
def remove_outliers(df, target: str, method='ensemble', contamination=0.1, voting_threshold=0.5) -> pd.DataFrame:
    """
    Advanced outlier detection using multiple methods.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    target : str
        Target column name
    method : str
        'ensemble' (default): Combines multiple methods with voting
        'isolation_forest': Uses Isolation Forest only
        'lof': Uses Local Outlier Factor only
        'robust_statistical': Uses Modified Z-score with MAD
        'all_strict': All methods must agree (strictest)
    contamination : float
        Expected proportion of outliers (0.05-0.2 typical)
    voting_threshold : float
        For ensemble method, fraction of methods that must flag as outlier (0.5 = majority)
    
    Returns:
    --------
    pd.DataFrame
        Cleaned dataframe without outliers
    """
    
    # Remove zero values first (domain-specific)
    df_nonzero = df[df[target] != 0].copy()
    
    if len(df_nonzero) < 10:
        print(f"Warning: Only {len(df_nonzero)} samples. Skipping outlier detection.")
        return df_nonzero
    
    # Prepare features: numerical columns + target
    numerical_cols = df_nonzero.select_dtypes(include=[np.number]).columns.tolist()
    
    # Scale features for better outlier detection
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_nonzero[numerical_cols])
    
    # Initialize outlier flags
    outlier_flags = {}
    
    # Method 1: Isolation Forest (excellent for high-dimensional data)
    if method in ['ensemble', 'isolation_forest', 'all_strict']:
        iso_forest = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_estimators=200,
            max_samples='auto',
            bootstrap=True
        )
        iso_predictions = iso_forest.fit_predict(X_scaled)
        outlier_flags['isolation_forest'] = (iso_predictions == -1)
        
    # Method 2: Local Outlier Factor (density-based, good for local anomalies)
    if method in ['ensemble', 'lof', 'all_strict']:
        n_neighbors = min(20, len(df_nonzero) - 1)
        lof = LocalOutlierFactor(
            n_neighbors=n_neighbors,
            contamination=contamination
        )
        lof_predictions = lof.fit_predict(X_scaled)
        outlier_flags['lof'] = (lof_predictions == -1)
    
    # Method 3: Modified Z-score with MAD (robust to outliers themselves)
    if method in ['ensemble', 'robust_statistical', 'all_strict']:
        target_values = df_nonzero[target].values
        median = np.median(target_values)
        mad = np.median(np.abs(target_values - median))
        
        # Modified Z-score (more robust than standard Z-score)
        if mad != 0:
            modified_z_scores = 0.6745 * (target_values - median) / mad
            # Threshold of 3.5 is standard for modified Z-score
            outlier_flags['robust_statistical'] = np.abs(modified_z_scores) > 3.5
        else:
            outlier_flags['robust_statistical'] = np.zeros(len(df_nonzero), dtype=bool)
    
    # Method 4: Multivariate Z-score on target (additional check)
    if method in ['ensemble', 'all_strict']:
        target_scaled = scaler.fit_transform(df_nonzero[[target]])
        outlier_flags['z_score'] = np.abs(target_scaled.flatten()) > 3
    
    # Combine methods based on selected strategy
    if method == 'ensemble':
        # Voting: flag as outlier if voting_threshold fraction of methods agree
        outlier_matrix = np.column_stack(list(outlier_flags.values()))
        votes = outlier_matrix.sum(axis=1)
        is_outlier = votes >= (len(outlier_flags) * voting_threshold)
        
    elif method == 'all_strict':
        # All methods must agree (most conservative)
        outlier_matrix = np.column_stack(list(outlier_flags.values()))
        is_outlier = outlier_matrix.all(axis=1)
        
    elif method in ['isolation_forest', 'lof', 'robust_statistical']:
        # Single method
        is_outlier = outlier_flags[method]
    
    else:
        raise ValueError(f"Unknown method: {method}")
    
    # Filter out outliers
    df_clean = df_nonzero[~is_outlier].copy()
    
    # Report
    n_outliers = is_outlier.sum()
    pct_removed = (n_outliers / len(df_nonzero)) * 100
    print(f"  → Removed {n_outliers}/{len(df_nonzero)} outliers ({pct_removed:.1f}%) using {method}")
    
    if method == 'ensemble' and len(outlier_flags) > 1:
        for flag_name, flags in outlier_flags.items():
            print(f"     • {flag_name}: {flags.sum()} outliers")
    
    return df_clean

In [20]:
def generate_synthetic_data(df_real, predictor1, predictor2, target, n_synthetic=20):
    X_real = df_real[[predictor1, predictor2]].values
    y_real = df_real[target].values
    
    np.random.seed(42)
    X_synthetic = []
    y_synthetic = []
    
    for _ in range(n_synthetic):
        idx1, idx2 = np.random.choice(len(df_real), 2, replace=False)
        alpha = np.random.beta(2, 2)
        
        x_new = alpha * X_real[idx1] + (1 - alpha) * X_real[idx2]
        x_new *= np.random.uniform(0.99, 1.01, size=2)
        
        ratio1 = x_new[0] / X_real[idx1, 0] if X_real[idx1, 0] != 0 else 1
        ratio2 = x_new[1] / X_real[idx1, 1] if X_real[idx1, 1] != 0 else 1
        y_new = y_real[idx1] * (ratio1**0.6) * (ratio2**0.4)
        y_new *= np.random.uniform(0.99, 1.01)
        
        X_synthetic.append(x_new)
        y_synthetic.append(y_new)
    
    df_synthetic = pd.DataFrame(X_synthetic, columns=[predictor1, predictor2])
    df_synthetic[target] = y_synthetic
    
    return pd.concat([df_real[[predictor1, predictor2, target]], df_synthetic], ignore_index=True)

def train_model_tunnels(df_clean, predictor_name, hue_name, target_name):
    predictor1, predictor2 = predictor_name, hue_name
    
    df_augmented = generate_synthetic_data(df_clean, predictor1, predictor2, target_name, n_synthetic=200)
    
    X = df_augmented[[predictor1, predictor2]].copy()
    X[predictor1 + '_LOG'] = np.log1p(X[predictor1])
    X[predictor2 + '_LOG'] = np.log1p(X[predictor2])
    y = df_augmented[target_name].astype(float)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model = TransformedTargetRegressor(regressor=LinearRegression(), func=np.log1p, inverse_func=np.expm1)
    model.fit(X_scaled, y)
    
    X_real = df_clean[[predictor1, predictor2]].copy()
    X_real[predictor1 + '_LOG'] = np.log1p(X_real[predictor1])
    X_real[predictor2 + '_LOG'] = np.log1p(X_real[predictor2])
    y_real = df_clean[target_name].astype(float)
    
    X_real_scaled = scaler.transform(X_real)
    y_pred = model.predict(X_real_scaled)
    
    r2 = r2_score(y_real, y_pred)
    mape = np.mean(np.abs((y_real - y_pred) / y_real)) * 100
    
    print(f"\nR² = {r2:.4f} | MAPE = {mape:.2f}%")
    
    class ScaledModel:
        def __init__(self, model, scaler, feature_names):
            self.model, self.scaler, self.feature_names = model, scaler, feature_names
        def predict(self, X_new):
            return self.model.predict(self.scaler.transform(X_new[self.feature_names]))
    
    return X_real, y_real, y_pred, ScaledModel(model, scaler, X.columns.tolist())
    

In [21]:
def train_and_calculate_metrics_tunnels(df, target_column, predictors):
    predictor1, predictor2 = predictors
    df_item = df[df[target_column] > 0][[predictor1, predictor2, target_column]].dropna()
    
    X, y, y_pred, model = train_model_tunnels(df_item, predictor1, predictor2, target_column)
    
    print(pd.DataFrame({
        predictor1: df_item[predictor1].values,
        predictor2: df_item[predictor2].values,
        'Actual': y.values,
        'Predicted': y_pred,
        'Error%': ((y.values - y_pred) / y.values * 100)
    }))
    
    return {target_column: {'X': X, 'y': y, 'y_predicted': y_pred, 'trained_model': model}}    

target_column = '9 - TÚNELES'
predictors = ['TUNELES UND', 'TUNELES KM']

df = df_vp[['LONGITUD KM', 'ALCANCE']].join(df_vp.loc[:, '1 - TRANSPORTE':'16 - DIRECCIÓN Y COORDINACIÓN'])
df[predictors[0]] = df_vp[predictors[0]]
df[predictors[1]] = df_vp[predictors[1]]

results = train_and_calculate_metrics_tunnels(df, target_column, predictors)    


R² = 0.8339 | MAPE = 9.76%
   TUNELES UND  TUNELES KM        Actual     Predicted     Error%
0            5      0.9200  6.644646e+07  6.562381e+07   1.238065
1            1      8.6000  1.632824e+08  1.275371e+08  21.891687
2           15      2.2747  1.906902e+08  2.023911e+08  -6.136095
